In [2]:
#Import your libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pandas.plotting import parallel_coordinates
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [3]:
file_url = r'C:\Users\Michel\git2\Ironhack-DAFT-Project7-ML-Netflix\imdb_processed_clean_encoded.csv'


In [4]:
df = pd.read_csv(file_url)

In [5]:
df.

,year,kind,rating,vote,runtime,genre1,genre2,country1,country2,language1,language2,cast1,cast2,director1,director2,composer1,composer2,writer1,writer2
0,2003.0,2,7.7,474.0,50.000000,6,2,62,71,16,110,817,4942,3924,611,2512,762,3299,2469
1,1994.0,7,8.1,18.0,83.000000,8,27,62,71,16,110,3599,381,3660,611,2512,762,4765,3786
2,1992.0,1,5.5,93.0,96.828211,0,20,37,71,33,110,2472,1621,4068,611,354,762,2582,1672
3,2004.0,1,5.3,13432.0,60.000000,6,4,62,71,16,37,3014,1499,4013,39,410,762,4877,288
4,1991.0,1,4.5,2177.0,96.000000,4,13,62,71,16,110,589,3310,1100,611,265,762,2868,405


In [82]:
df[df['rating']>9.5]

,year,kind,rating,vote,runtime,genre1,genre2,country1,country2,language1,language2,cast1,cast2,director1,director2,composer1,composer2,writer1,writer2
3138,2004.0,0,9.6,8.0,30.0,17,27,63,71,62,110,3425,3570,1314,611,2512,762,4765,3786


In [6]:
target='rating'

In [28]:
x=df.drop(columns=target)
#x=df_enc.drop(index=[712],columns='survived')
#x_check=titanic_enc.tail(1).drop(columns='survived')
y=round(df[target]).astype(int)

In [29]:
y

0       8
1       8
2       6
3       5
4       4
       ..
8064    4
8065    6
8066    5
8067    6
8068    8
Name: rating, Length: 8069, dtype: int32

In [12]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

In [22]:
y.head()

0    8.0
1    8.0
2    6.0
3    5.0
4    4.0
Name: rating, dtype: float64

In [31]:
#METHOD 1:
rfe_selector = RFE(estimator=RandomForestClassifier(),n_features_to_select = 5 , step = 1, verbose=1)
m=rfe_selector.fit(x, y)

Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.


In [32]:
x.columns[m.get_support()]
print("Num Features: %s" % (m.n_features_))
print("Selected Features: %s" % (m.support_))
print("Feature Ranking: %s" % (m.ranking_))

Num Features: 5
Selected Features: [False False  True False False False False False False False  True  True
  True False False False  True False]
Feature Ranking: [ 3  9  1  2  6  7  8 11 10 12  1  1  1 14  4 13  1  5]


In [33]:
x.loc[:, m.support_].columns

Index(['vote', 'cast1', 'cast2', 'director1', 'writer1'], dtype='object')

In [36]:
#METHOD 2:
from sklearn.feature_selection import SelectFromModel
SFM = SelectFromModel(estimator=RandomForestClassifier())
s=SFM.fit(x, y)

In [38]:
n_features = s.transform(x).shape[1]
n_features

10

In [40]:
feature_idx = s.get_support()
feature_name = x.columns[feature_idx]
feature_name 

Index(['year', 'vote', 'runtime', 'genre1', 'cast1', 'cast2', 'director1',
       'composer1', 'writer1', 'writer2'],
      dtype='object')

In [44]:
#METHOD 3:
from sklearn.feature_selection import RFECV

In [65]:
y.value_counts()

7     2530
8     2081
6     1910
5      703
4      395
9      272
3      132
2       45
10       1
Name: rating, dtype: int64

In [62]:
rfecv = RFECV(
    estimator=RandomForestClassifier(),
    min_features_to_select=2,
    step=5,
    n_jobs=-1, #multiprocessing: -1: use all available processor
    scoring="r2",
    cv=5,
    verbose=1
)

In [61]:
y.describe()

count    8069.000000
mean        6.674557
std         1.309616
min         2.000000
25%         6.000000
50%         7.000000
75%         8.000000
max        10.000000
Name: rating, dtype: float64

In [63]:
m= rfecv.fit(x, y)

C:\Users\Michel\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [ ]:
METHOD 3:
from sklearn.feature_selection import RFECV

rfecv = RFECV(
    estimator=RandomForestClassifier(),
    min_features_to_select=2,
    step=5,
    n_jobs=-1, #uses all available processors
    scoring="r2",
    cv=5,)
m= rfecv.fit(x, y)

X.columns[rfecv.support_]

TRAIN_TEST_SPLIT:
from sklearn.model_selection import train_test_split
y = netflix['ranking']
x = netflix[features] # features found thanks to feature selector

X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=5)




HYPERPARAMETER TUNING
METHOD 1 GRID SEARCH :
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
grid_search = GridSearchCV(AdaBoostClassifier(random_state=0), #take a model like RFC responsible for searching
                           {
                              'n_estimators':np.arange(5,400,5)
                            
                            },cv=5, scoring="accuracy",verbose=1,n_jobs=-1 #my goal is Accuracy , we want verbosity for more details
                           ) #cv = crossvalidation, how many folds you want
grid_search.fit(X_train,y_train)

grid_search.best_params_
grid_search.best_score_

METHOD 1 RANDOM SEARCH:
rand_search = RandomizedSearchCV(RandomForestClassifier(random_state=0),
                           {
                              'n_estimators':np.arange(5,400,5)
                            
                            },cv=5, scoring="accuracy",verbose=1,n_jobs=-1
                           )
rand_search.fit(X_train,y_train)


rand_search.best_params_
rand_search.best_score_


CHOICE OF MODEL : 



From sklearn.ensemble import AdaBoostClassifier

clf= AdaBoostClassifier(n_estimators=100, random_state=0)

	n_estimator found thanks to hyperparameter tuning

clf.fit(X,y)

clf.predict(x_train)

clf.score(X,y)
